# Chemical equilibrium with fixed pH

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 7th, 2022</p>

Fixing the pH of an aqueous solution is a common procedure in biogeochemical modeling. This requires, however, the introduction of a new *degree of freedom* in the problem. This is because we cannot, in general, obtain a desired pH without titrating a substance in the solution. Thus, in the example given below, the system will be open to H<sup>+</sup>. The unknown amount of H<sup>+</sup> necessary to enter or leave the system to attain the requested pH is the new degree of freedom, and is calculated along with the amounts of species in equilibrium.

```{note}
The choice of H<sup>+</sup> as the *titrant* when fixing the pH of an aqueous solution may seem arbitrary and unintuitive to some coming from a different background or experienced with another convention used by other chemical modeling codes. For example, it can be questioned that a charged species not found isolated in nature (compared to HCl, for example) is chosen as the titrant. However, there are mathematical reasons behind this choice that make the problem to be solved more simply and efficiently. We will give examples later on how to combine this pH constraint with other conditions to demonstrate the full potential of this convention.
```

For our example, let's create a {{ChemicalSystem}} object containing a simple aqueous phase (with thermodynamic data for the species fetched from a {{PhreeqcDatabase}} object):

In [44]:
from reaktoro import *

db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase("H2O H+ OH- Na+ Cl- HCO3- CO2 CO3-2")
solution.setActivityModel(ActivityModelPitzerHMW())

system = ChemicalSystem(db, solution)

Let's create a chemical equilibrium solver that expects the following properties as input conditions:

* temperature;
* pressure; and
* pH.

The code below will first create an {{EquilibriumSpecs}} object in which the above properties are marked as inputs and then use that to create our {{EquilibriumSolver}} object:

In [45]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

solver = EquilibriumSolver(specs)

We create now an initial chemical state for our system (using {{ChemicalState}}), which will represent a 1 molal NaCl solution with 0.4 molal dissolved CO{{_2}}:

In [46]:
state = ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.4, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+---------+------+
| Property        |   Value | Unit |
+-----------------+---------+------+
| Temperature     |  303.15 |    K |
| Pressure        |  101325 |   Pa |
| Charge:         |  -3e-16 |  mol |
| Element Amount: |         |      |
| :: H            | 111.012 |  mol |
| :: C            |     0.4 |  mol |
| :: O            | 56.3062 |  mol |
| :: Na           |       1 |  mol |
| :: Cl           |       1 |  mol |
| Species Amount: |         |      |
| :: H2O          | 55.5062 |  mol |
| :: H+           |   1e-16 |  mol |
| :: OH-          |   1e-16 |  mol |
| :: Na+          |       1 |  mol |
| :: Cl-          |       1 |  mol |
| :: HCO3-        |   1e-16 |  mol |
| :: CO2          |     0.4 |  mol |
| :: CO3-2        |   1e-16 |  mol |
+-----------------+---------+------+


We need now to specify the desired values for temperature, pressure, and pH in the equilibrium state we want to compute. This is done next using {{EquilibriumConditions}}:

In [47]:
conditions = EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "atm")
conditions.pH(1.0)

We are now ready for our chemical equilibrium calculation:

In [48]:
solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        | 1.01325e+06 |   Pa |
| Charge:         |    0.122916 |  mol |
| Element Amount: |             |      |
| :: H            |     111.135 |  mol |
| :: C            |         0.4 |  mol |
| :: O            |     56.3062 |  mol |
| :: Na           |           1 |  mol |
| :: Cl           |           1 |  mol |
| Species Amount: |             |      |
| :: H2O          |     55.5062 |  mol |
| :: H+           |    0.122921 |  mol |
| :: OH-          | 9.03813e-13 |  mol |
| :: Na+          |           1 |  mol |
| :: Cl-          |           1 |  mol |
| :: HCO3-        | 4.36986e-06 |  mol |
| :: CO2          |    0.399996 |  mol |
| :: CO3-2        | 1.81834e-14 |  mol |
+-----------------+-------------+------+


The printed state above does not show pH. We can obtain aqueous properties from a chemical state using {{AqueousProps}}:

In [51]:
aprops = AqueousProps(state)

print("AQUEOUS PROPERTIES AT EQUILIBRIUM")
print(aprops)

AQUEOUS PROPERTIES AT EQUILIBRIUM
+--------------------------+-------------+-------+
| Property                 |       Value |  Unit |
+--------------------------+-------------+-------+
| Temperature              |      323.15 |     K |
| Pressure                 | 1.01325e+06 |    Pa |
| Ionic Strength (Effect.) |     1.06151 | molal |
| Ionic Strength (Stoich.) |     1.06151 | molal |
| pH                       |           1 |       |
| pE                       |      5.1853 |       |
| Eh                       |    0.332481 |     V |
| Element Molality:        |             |       |
| :: C                     |    0.400016 | molal |
| :: Na                    |     1.00004 | molal |
| :: Cl                    |     1.00004 | molal |
| Species Molality:        |             |       |
| :: H+                    |    0.122926 | molal |
| :: OH-                   |  9.0385e-13 | molal |
| :: Na+                   |     1.00004 | molal |
| :: Cl-                   |     1.00004 | molal

In the table above, we see that temperature, pressure, and pH have values exactly like those given earlier in the {{EquilibriumConditions}} object (although with different temperature and pressure units).

As a result of enforcing pH via `specs.pH()`, this executed equilibrium calculation considered the system to be open to H<sup>+</sup>. Let's find out below how much H<sup>+</sup> entered/leaved the system so that the desired pH could be reached:

In [54]:
print("Amount of H+ that entered the system:", state.equilibrium().implicitTitrantAmounts()[0], "mol")

Amount of H+ that entered the system: 0.12291636480068265 mol


Since H<sup>+</sup> is a charged species, our final chemical state does not have zero charge as the initial state:

In [53]:
print("Charge at equilibrium state:", state.charge(), "mol")

Charge at equilibrium state: 0.122916 mol


We'll learn in the next tutorial how a pH constraint can be combined with a charge constraint so that the system has zero charge at equilibrium (or whatever other value you may want to impose).